# Ein- und Ausgabe am Beispiel des Ising-Modells

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Dieses Jupyter-Notebook basiert auf dem Jupyter-Notebook [4-04-Ising-Modell.ipynb](4-04-Ising-Modell.ipynb) zum Ising-Modell und demonstriert das Schreiben in und das Lesen aus Dateien. Im Folgenden werden nur die neuen Codeteile kommentiert. Für die Beschreibung des Codes zum Ising-Modell sei auf das Jupyter-Notebook [4-04-Ising-Modell.ipynb](4-04-Ising-Modll.ipynb) verwiesen.

## Importanweisungen

Im Zusammenhang mit der Ein- und Ausgabe benötigen wir hier einige zusätzliche Importanweisungen. Zunächst wird der Namensraum des `csv`-Moduls aus der Python-Standardbibliothek importiert. Wir werden dieses Modul benutzen, um CSV-Dateien zu schreiben. Ferner wird aus dem `pathlib`-Modul der Python-Standardbibliothek die Klasse `Path` importiert, die den Umgang mit Pfadnamen erleichtert. Schließlich importieren wir noch `pandas`, ein umfangreiches Paket zur Datenanalyse. Es ist üblich, hierfür die Abkürzung `pd` einzuführen, so wie `numpy` mit `np` abgekürzt wird. Wir werden die pandas-Bibliothek lediglich dazu benutzen, um Daten im Jupyter-Notebook in einfacher Weise in Tabellenform darzustellen.

In [ ]:
from collections import defaultdict
import csv
from math import exp
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact_manual
from IPython.display import display, HTML

plt.style.use("numphyspy.style")

## Temperaturabhängigkeit von spezifischer Wärme und magnetischer Suszeptibilität des Ising-Modells

### Name des Notebooks als Basisname für Datenfiles und des Unterverzeichnisses für Daten

Zu Beginn legen wir zwei Parameter fest und verwenden dabei durchgehend groß geschriebene Variablennamen um anzudeuten, dass diese Variablen in folgenden Codezellen nicht verändert werden sollten. `NB_BASENAME` bezieht sich auf den Basisnamen, aus dem später die Dateinamen konstruiert werden. `DATADIR` ist das Verzeichnis, in dem Dateien abgelegt werden. Es handelt sich dabei um ein Unterverzeichnis des Verzeichnisses, in dem sich das Jupyter-Notebook befindet. Bei Bedarf können diese beiden Parameter hier angepasst werden.

In [ ]:
NB_BASENAME = "Ising-Modell-IO"
DATADIR = Path.cwd() / "data"

### Bestimmung der nächsten Nachbarn

In [ ]:
def neighbours(size, dimension):
    neighbour = np.empty((size**dimension, 2*dimension),
                         dtype=int)
    index_shape = (size,)*dimension
    index = np.arange(size**dimension).reshape(index_shape)
    column = 0
    for axis in range(dimension):
        for shift in (-1, 1):
            neighbour[:, column] = np.roll(index, shift,
                                           axis=axis
                                           ).flatten()
            column = column + 1
    return neighbour

### Erzeugung des Anfangszustands

In [ ]:
rng = np.random.default_rng(123456)

def initial_state(n_spins, neighbour):
    state = rng.choice((-1, 1), size=n_spins)
    magnetization = np.sum(state)
    sum_nbr_spins = np.sum(state[neighbour], axis=1)
    energy = -np.sum(state*sum_nbr_spins)/2
    return state, energy, magnetization

### Iterationsschritt im Wolff-Algorithmus

In [ ]:
def wolff_generator(size, dimension, beta, full=False):
    n_spins = size**dimension
    neighbour = neighbours(size, dimension)
    p_limit = 1-exp(-2*beta)
    state, energy, magnetization = initial_state(n_spins,
                                                 neighbour)
    while True:
        if full:
            yield state, energy, magnetization
        else:
            yield energy, magnetization
        n_start = rng.choice(n_spins)
        to_do = {n_start}
        cluster = {n_start}
        state[n_start] = -state[n_start]
        while to_do:
            n_spin = to_do.pop()
            for nbr in neighbour[n_spin, :]:
                if (state[nbr] != state[n_spin]
                        and np.random.uniform() < p_limit):
                    to_do.add(nbr)
                    cluster.add(nbr)
                    state[nbr] = -state[nbr]
        delta_m = 2*state[n_start]*len(cluster)
        magnetization = magnetization + delta_m
        delta_e = 0
        for n_spin in cluster:
            sum_nbr_spins = 0
            for n_nbr_spin in neighbour[n_spin, :]:
                if n_nbr_spin not in cluster:
                    sum_nbr_spins += state[n_nbr_spin]
            delta_e = delta_e - 2*state[n_spin]*sum_nbr_spins
        energy = energy + delta_e

### Energie, Magnetisierung sowie deren erste und zweite Momente

In [ ]:
def thermo_values(size, dimension, beta, n_steps_init,
                  n_steps, absmagn):
    state = wolff_generator(size, dimension, beta)
    for n in range(n_steps_init):
        next(state)
    e_sum = 0
    e2_sum = 0
    m_sum = 0
    m2_sum = 0
    for n, (energy, magnetization) in zip(range(n_steps),
                                          state):
        e_sum = e_sum + energy
        e2_sum = e2_sum + energy**2
        if absmagn:
            magnetization = abs(magnetization)
        m_sum = m_sum + magnetization
        m2_sum = m2_sum + magnetization**2
    n_spins = size**dimension
    spec_heat = e2_sum/n_steps-(e_sum/n_steps)**2
    spec_heat = spec_heat * beta**2 / n_spins
    magn_susc = m2_sum/n_steps-(m_sum/n_steps)**2
    magn_susc = magn_susc * beta / n_spins
    return 1/beta, spec_heat, magn_susc

### Erzeugung von Dateinamen

Wenn man Ergebnisse zu unterschiedlichen Parametersätzen in Dateien schreibt, möchte man vermeiden, schon existierende Dateien zu überschreiben. Daher muss man einen neue Dateinamen erzeugen. In unserem Fall sollen Dateinamen aus einem Basisnamen `basename` und einer nach einem Unterstrich angehängten Ziffernfolge sowie einer Dateinamenserweiterung `extension` bestehen. Die Aufgabe besteht also darin, eine noch nicht existierende Ziffernfolge zu erzeugen. Dazu wird zunächst mit Hilfe der Funktion `search_files` eine sortierte Liste der existierenden Dateinamen mit dem vorgegebenen Dateinamensformat erzeugt. Der letzte Eintrag in der Liste entspricht der höchsten, bereits existierenden laufenden Nummer, die in `max_nr` abgelegt wird. Sind noch keine Dateien vorhanden, erhält `max_nr` den Wert `-1`. Indem man `max_nr` um eins erhöht, erhält man die laufende Nummer, aus der man den nächsten Dateinamen konstruieren kann. Dieser wird abschließend an das vorgegebene Verzeichnis `dir` angehängt.

In [ ]:
def search_files(dir, basename, extension, nr_of_digits=4):
    filenamepattern = "".join([basename, "_",
                               "[0-9]"*nr_of_digits, ".",
                               extension])
    return sorted(dir.glob(filenamepattern))

def next_filename(dir, basename, extension, nr_of_digits=4):
    existing_files = search_files(dir, basename, extension,
                                  nr_of_digits)
    if len(existing_files):
        latest_file = existing_files[-1]
        max_nr = int(latest_file.stem[-nr_of_digits:])
    else:
        max_nr = -1
    nextfilename = "".join([basename, "_",
                            f"{max_nr+1:0{nr_of_digits}}",
                            ".", extension])
    return dir / nextfilename

### Schreiben von Daten

Das Schreiben der Daten in eine Datei soll im CSV-Format erfolgen, wobei hier die einzelnen Einträge in einer Zeile durch Kommas getrennt werden. Das Schreiben erfolgt in einem `with`-Kontext, der sicherstellt, dass die Datei auf jeden Fall korrekt geschlossen wird. Zunächst werden die Werte der in der Rechnung verwendeten Parameter in die Datei geschrieben. Die betreffenden Zeilen beginnen mit einem `#` um anzudeuten, dass es sich hier nicht um die eigentlichen Daten handelt. Um die Angabe der Parameter von der Überschrift, die die Bedeutung der Spalten angibt, abzugrenzen, wird noch eine mit einem `#` beginnende Leerzeile eingefügt. Abschließend erfolgt die Schleife über die Werte der inversen Temperatur, wobei in jedem Durchlauf eine neue Datenzeile geschrieben wird. Diese enthält die Temperatur, die spezifische Wärme je Spin und die dynamische Suszeptibilität je Spin.

In [ ]:
def write_c_chi_of_temp(size, dimension, temperature_range,
                        n_T, log_n_steps_init, log_n_steps,
                        absmagn, filename):
    n_steps_init = 10**log_n_steps_init
    n_steps = 10**log_n_steps
    with open(filename, "w", encoding="utf-8", newline=""
              ) as csvfile:
        csvwriter = csv.writer(csvfile)
        for k in ("dimension", "size", "n_steps_init",
                  "n_steps", "absmagn"):
            csvwriter.writerow((f"# {k} = {locals()[k]}",))
        csvwriter.writerow("#")
        if absmagn:
            csvwriter.writerow(("# T", "C/N", "χ/N"))
        else:
            csvwriter.writerow(("# T", "C/N", "χ'/N"))
        beta_values = 1/np.linspace(*temperature_range, n_T)
        for beta in beta_values:
            result = thermo_values(
                size, dimension, beta, n_steps_init,
                n_steps, absmagn)
            csvwriter.writerow(result)

### Berechtigung zum Schreiben in das Datenverzeichnis

Neben der Einstellung von Parametern im Zusammenhang mit dem Ising-Modell kommt hier noch die Abfrage hinzu, ob in das Unterverzeichnis `data` geschrieben werden darf oder dieses, falls es noch nicht existiert, neu angelegt werden darf. Um die Erlaubnis zu erteilen, muss das entsprechende Kästchen angeklickt werden.

In [ ]:
if Path.exists(DATADIR):
    info_text = ("Ein Unterverzeichnis data existiert. "
                 "Bestätigen Sie, dass dort Daten "
                 "abgelegt werden dürfen.")
else:
    info_text = ("Bestätigen Sie, dass ein "
                 "Unterverzeichnis data angelegt werden "
                 "darf, um dort Daten abzulegen.")


wide_label = {"description_width": "initial"}
widget_dict = {"size":
               widgets.IntSlider(
                   value=100, min=100, max=1000, step=100,
                   description=r"$n_\text{spins}$"),
               "dimension":
               widgets.RadioButtons(
                   options=[("eindimensional", 1),
                            ("zweidimensional", 2)],
                   value=1, description="Dimension"),
               "absmagn":
               widgets.RadioButtons(
                   options=[("M", False),
                            ("|M|", True)],
                   value=False,
                   description="Magnetisierung"),
               "temperature_range":
               widgets.FloatRangeSlider(
                   value=(0.05, 5), min=0.05, max=5,
                   step=0.05,
                   description="Temperaturbereich",
                   style=wide_label),
               "n_T":
               widgets.IntSlider(
                   value=50, min=10, max=200, step=10,
                   description="Temperaturwerte",
                   style=wide_label),
               "log_n_steps_init":
               widgets.IntSlider(
                   value=3, min=1, max=7, step=1,
                   description=r"$\log_{10}"
                               r"(n_\text{init})$"),
               "log_n_steps":
               widgets.IntSlider(
                   value=4, min=3, max=9, step=1,
                   description=r"$\log_{10}"
                               r"(n_\text{steps})$"),
               "datadir_perm":
               widgets.Checkbox(
                   value=False, description=info_text,
                   indent=False,
                   layout=widgets.Layout(width="100%"))
               }

def handle_dim_change(change):
    attributes = {1: {"max": 1000, "min": 50, "step": 50,
                      "value": 100},
                  2: {"step": 5, "min": 5, "max": 40,
                      "value": 20}}
    for k, v in attributes[change.new].items():
        setattr(widget_dict["size"], k, v)

widget_dict["dimension"].observe(handle_dim_change,
                                 names="value")

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

In [ ]:
@interact_start(**widget_dict)
def generate_and_write_data(size, dimension,
                            temperature_range, n_T,
                            log_n_steps_init, log_n_steps,
                            absmagn, datadir_perm):
    if not datadir_perm:
        return
    if not Path.exists(DATADIR):
        Path.mkdir(DATADIR)
    filename = next_filename(DATADIR, NB_BASENAME, "csv")
    write_c_chi_of_temp(size, dimension, temperature_range,
                        n_T, log_n_steps_init, log_n_steps,
                        absmagn, filename)

### Einlesen von Daten

Die folgenden drei Funktionen werden im Zusammenhang mit dem Einlesen von Daten verwendet. Da die für die Rechnung verwendeten Parameter nicht im Dateinamen enthalten sind, hilft die Funktion `get_parameters`, die richtige Datei zu identifizieren. Dazu liest diese Funktion die in einer Datei `filename` verwendeten Parameter aus den mit `#` beginnenden Zeilen ein und gibt diese im Dictionary `parameter_info` zurück. Die Funktion `parameter_dataframe` verwendet diese Informationen und gibt sie zusammen mit den zugehörigen Dateinamen in einem DataFrame von Pandas zurück. In dieser Form ist später eine Darstellung der Informationen als Tabelle leicht möglich. Die Daten sowie insbesondere die Variable `absmagn`, die angibt, ob der Absolutbetrag der Magnetisierung bei der Berechnung der magnetischen Suszeptibilität verwendet wurde, werden beim Aufruf der Funktion `get_data` zurückgegeben. Dabei wird der Dateiname durch das Attribut `value` des noch zu definierenden Auswahl-Widgets `fileselector` festgelegt.

In [ ]:
def get_parameters(filename):
    parameter_info = dict()
    with open(filename) as file:
        for line in file:
            if line.strip() == "#":
                break
            else:
                param_line = line.lstrip("# ").rstrip()
                k, v = param_line.split(" = ")
                parameter_info[k] = v
    return parameter_info

def parameter_dataframe(filenames, keys, nr_of_digits=4):
    data_dict = defaultdict(list)
    for filename in filenames:
        file_index = filename.stem[-nr_of_digits:]
        data_dict["Dateinummer"].append(file_index)
        parameter_dict = get_parameters(filename)
        for k in keys:
            data_dict[k].append(parameter_dict[k])
    return pd.DataFrame(data_dict)

def get_data():
    try:
        filename = fileselector.value
    except NameError:
        raise ValueError("bitte wählen Sie eine Datei aus")
    parameter_dict = get_parameters(filename)
    size = int(parameter_dict["size"])
    dimension = int(parameter_dict["dimension"])
    absmagn = parameter_dict["absmagn"]
    data = np.loadtxt(fileselector.value, delimiter=",")
    return data, size, dimension, absmagn

### Darstellung der Parameter für die vorhandenen Dateien

Das von `parameter_dataframe` zurückgegebene Dataframe wird als HTML-Tabelle dargestellt.

In [ ]:
existing_files = search_files(DATADIR, NB_BASENAME, "csv")
column_names = ["dimension", "size", "n_steps_init",
                "n_steps", "absmagn"]
df = parameter_dataframe(existing_files[::-1], column_names)
HTML(df.to_html(index=False))

### Widget für die Dateiauswahl

Hier wird das Auswahl-Widget definiert, mit dessen Hilfe die einzulesende Datei ausgewählt wird.

In [ ]:
existing_files = search_files(DATADIR, NB_BASENAME, "csv")
try:
    fileselector = widgets.Select(
        options=existing_files[::-1],
        value=existing_files[-1],
        description="Dateien:",
        layout=widgets.Layout(width="100%")
    )
except IndexError:
    print("aktuell sind keine Dateien verfügbar")
else:
    display(fileselector)

### Graphische Darstellung der Daten

Nachdem die Daten mit Hilfe der Funktion `get_data` eingelesen wurden, werden sie graphisch dargestellt.

In [ ]:
data, size, dimension, absmagn = get_data()

temperature = data[:, 0]
spec_heat = data[:, 1]
magn_susc = data[:, 2]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.set_xlabel("$T$")
ax1.set_ylabel("$C/N$")
ax1.plot(temperature, spec_heat)

ax2.set_xlabel("$T$")
if absmagn == "True":
    ax2.set_ylabel(r"$\chi'/N$")
else:
    ax2.set_ylabel(r"$\chi/N$")
ax2.plot(temperature, magn_susc)